In [1]:
import gc
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
from time import time
from datetime import date
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

%matplotlib inline

In [2]:
pd.set_option('display.max_columns', 1000)

# 1 数据预处理

In [3]:
# 载入数据
order = pd.read_csv("../../data/level2/m111-order-sku.csv", sep=',', parse_dates=['order_date'])
dis = pd.read_csv("../../data/level2/m111-dis-sku.csv", sep=',', parse_dates=['dis_date'])
inv = pd.read_csv(
    "../../data/level2/m111-inv-sku.csv", sep=',', parse_dates=['period_wid']
).rename(columns={'period_wid': 'inv_date'})
category = pd.read_csv(
    "../../data/level2/m111-item-category.csv", sep=','
).rename(columns={'sales_segment1_code': 'category'})

## 1.1 处理订单数据

In [4]:
# 取2017年至2018年区间中的数据
order = order.loc[(order['order_date'] >= '2017-01-01') & (order['order_date'] <= '2018-12-31')]

In [5]:
# 删除订单量为0的记录
order = order.loc[~(order.qty == 0)]

In [6]:
# 删除异常值
order = order.join(category.set_index('item_code'), on='item_code', how='left').fillna('-1')
order = order.loc[~(order.category == 'TGWL')]
order = order.loc[~(order.category == -1)]
order = order.loc[~((order.category == 'CRYJ') & (order.qty > 10000))]
order = order.loc[~((order.category == 'CRZJ') & (order.qty > 10000))]
order = order.loc[~((order.category == 'CRZTCF') & (order.qty > 10000))]
order = order.loc[~((order.category == 'RR') & (order.qty > 10000))]

In [7]:
# 划分数据集
df_test = order.loc[order['order_date'] >= '2018-10-01']
order = order.loc[order['order_date'] <= '2018-09-30']

## 1.2 处理分销数据

In [10]:
# 取2017年至2018年区间中的数据（分销数据只有从3月份开始的数据且3月份不完整）>_<|||
dis = dis.loc[(dis['dis_date'] >= '2017-04-01') & (dis['dis_date'] <= '2018-09-30')]

In [11]:
# 处理分销量为负数的情况
dis['qty'] = dis['qty'].apply(lambda x: -x if x < 0 else x)

In [12]:
# 删除分销量为0的记录
dis = dis.loc[~(dis.qty == 0)]

In [13]:
# 删除异常值
dis = dis.join(category.set_index('item_code'), on='item_code', how='left').fillna('-1')
dis = dis.loc[~(dis.category == 'TGWL')]
dis = dis.loc[~(dis.category == '-1')]
dis = dis.loc[~((dis.category == 'CRYJ') & (dis.item_code == '12173000000014'))]
dis = dis.loc[~((dis.category == 'CRZTCF') & (dis.qty > 10000))]
dis = dis.loc[~((dis.category == 'RR') & (dis.item_code == '21099910000123'))]

In [14]:
dis_month = dis.copy()
dis_month['month'] = dis_month['dis_date'].astype('str').apply(lambda x: x[:7])
dis_month = dis_month.groupby(['item_code', 'month'])[['qty']].sum()
dis_month = dis_month.unstack(level=-1).fillna(0)
dis_month.columns = pd.date_range('2017-04-30', '2018-09-30', freq='M')
dis_month = dis_month.reindex(order_month.index).fillna(0)

## 1.3 处理库存数据

**问题**：
1. 按月进行统计的话，订单、提货、分销的数值很大，只要拟合不准，误差将是巨大的
2. 现在库存是取SKU最后一天的库存作为当月的库存，但是这样也有个问题——会有一些SKU在最后一天没有记录（但是如果以天作为预测粒度的话，就不存在这个问题了）

In [15]:
# 取2017年至2018年区间中的数据（库存数据只有从2017年6月份开始的数据）>_<|||
inv = inv.loc[(inv['inv_date'] >= '2017-06-01') & (inv['inv_date'] <= '2018-09-30')]

In [16]:
# 处理分销量为负数的情况
# inv['qty'] = inv['qty'].apply(lambda x: -x if x < 0 else x)

# 删除库存数据中小于等于0的记录
# 疑问：为什么会存在库存小于0的记录（入不敷出）？
inv = inv.loc[~(inv.qty <= 0)]

In [17]:
# 取每月的最后一天作为当月的库存
inv = inv.loc[inv['inv_date'].isin(pd.date_range('2017-06-30', '2018-09-30', freq='M'))]

In [18]:
inv = inv.join(category.set_index('item_code'), on='item_code', how='left').fillna('-1')

In [19]:
# 删除异常值
# inv = inv.loc[~(inv.qty > 1000000)]
inv = inv.loc[~(inv.category == '-1')]
inv = inv.loc[~(inv.category == 'TGWL')]
inv = inv.loc[~((inv.category == 'RR') & (inv.item_code == '21099910000123'))]
inv = inv.loc[~((inv.category == 'CRZTCF') & (inv.item_code == '21074110000008'))]
inv = inv.loc[~((inv.category == 'JSJ') & (inv.item_code == '21099910000164'))]
inv = inv.loc[~((inv.category == 'JSJ') & (inv.item_code == '21012500000009'))]

In [20]:
inv_month = inv.copy()
inv_month['month'] = inv_month['inv_date'].astype('str').apply(lambda x: x[:7])
inv_month = inv_month.groupby(['item_code', 'month'])[['qty']].sum()
inv_month = inv_month.unstack(level=-1).fillna(0)
inv_month.columns = pd.date_range('2017-06-30', '2018-09-30', freq='M')
inv_month = inv_month.reindex(order_month.index).fillna(0)

## 1.4 处理品类信息

In [21]:
# Pandas的bug：'21054110000024', '21054110000025', '21054110000084', '21054110000085' 这4个字符串居然是重复的，取消注释即可看到
# category.loc[category.item_code.duplicated()]

In [22]:
# 解决上面bug的下策，避免下面reindex报错
category.drop_duplicates(['item_code'], keep='first', inplace=True)

In [23]:
category = category.set_index('item_code').reindex(order_month.index)

In [24]:
encoder = LabelEncoder()
category['category'] = encoder.fit_transform(category['category'])

## 1.5 得到每个品类每个月的提货数据

In [25]:
order_cate_month = order_month.reset_index()
order_cate_month['category'] = category['category'].values
order_cate_month = order_cate_month.groupby('category')[order_month.columns].sum()

## 1.6 得到每个品类每个月的分销数据

In [26]:
dis_cate_month = dis_month.reset_index()
dis_cate_month['category'] = category['category'].values
dis_cate_month = dis_cate_month.groupby('category')[dis_month.columns].sum()

## 1.7 得到每个品类每个月的库存数据

In [27]:
inv_cate_month = inv_month.reset_index()
inv_cate_month['category'] = category['category'].values
inv_cate_month = inv_cate_month.groupby('category')[inv_month.columns].sum()

## 1.8 对所有数据取对数

In [28]:
order_month = np.log1p(order_month)
dis_month = np.log1p(dis_month)
inv_month = np.log1p(inv_month)

order_cate_month = np.log1p(order_cate_month)
dis_cate_month = np.log1p(dis_cate_month)
inv_cate_month = np.log1p(inv_cate_month)

# 2 特征工程

In [29]:
def prepare_dataset(order, dis, inv, year, month, is_train=True, name_prefix=None):
    X = {}
    
    # 提货的统计特征（28个特征）
    for i in [3, 6, 9, 12]:
        dt = date(year, month, 1)
        tmp = order[pd.date_range(end=dt, periods=i, freq='M')]  # 前i个月提货量
        X['ord_diff_mean_pre_%s' % i] = tmp.diff(axis=1).mean(axis=1).values  # 前i个月提货量的平均一阶差分
        X['ord_sum_decay_pre_%s' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values  # 前i个月提货量的和（带衰减）
        X['ord_mean_pre_%s' % i] = tmp.mean(axis=1).values  # 前i个月提货量的平均值
        X['ord_median_pre_%s' % i] = tmp.median(axis=1).values  # 前i个月提货量的中位数
        X['ord_max_pre_%s' % i] = tmp.max(axis=1).values  # 前i个月提货量的最大值
        X['ord_min_pre_%s' % i] = tmp.min(axis=1).values  # 前i个月提货量的最小值
        X['ord_std_pre_%s' % i] = tmp.std(axis=1).values  # 前i个月提货量的标准差
        
    # 分销的统计特征（21个特征）>_<|||
    for i in [3, 6, 9]:
        dt = date(year, month, 1)
        tmp = dis[pd.date_range(end=dt, periods=i, freq='M')]  # 前i个月分销量
        X['dis_diff_mean_pre_%s' % i] = tmp.diff(axis=1).mean(axis=1).values  # 前i个月分销量的平均一阶差分
        X['dis_sum_decay_pre_%s' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values  # 前i个月分销量的和（带衰减）
        X['dis_mean_pre_%s' % i] = tmp.mean(axis=1).values  # 前i个月分销量的均值
        X['dis_median_pre_%s' % i] = tmp.median(axis=1).values  # 前i个月分销量的中位数
        X['dis_max_pre_%s' % i] = tmp.max(axis=1).values  # 前i个月分销量的最大值
        X['dis_min_pre_%s' % i] = tmp.min(axis=1).values  # 前i个月分销量的最小值
        X['dis_std_pre_%s' % i] = tmp.std(axis=1).values  # 前i个月分销量的标准差
        
    # 库存的统计特征（14个）>_<|||
    for i in [3, 6]:
        dt = date(year, month, 1)
        tmp = inv[pd.date_range(end=dt, periods=i, freq='M')]  # 前i个月库存量
        X['inv_diff_mean_pre_%s' % i] = tmp.diff(axis=1).mean(axis=1).values  # 前i个月库存量的平均一阶差分
        X['inv_sum_decay_pre_%s' % i] = (tmp * np.power(0.9, np.arange(i)[::-1])).sum(axis=1).values  # 前i个月库存量的和（带衰减）
        X['inv_mean_pre_%s' % i] = tmp.mean(axis=1).values  # 前i个月库存量的均值
        X['inv_median_pre_%s' % i] = tmp.median(axis=1).values  # 前i个月库存量的中位数
        X['inv_max_pre_%s' % i] = tmp.max(axis=1).values  # 前i个月库存量的最大值
        X['inv_min_pre_%s' % i] = tmp.min(axis=1).values  # 前i个月库存量的最小值
        X['inv_std_pre_%s' % i] = tmp.std(axis=1).values  # 前i个月库存量的标准差
        
    # 提货天数特征（12个特征）
#     for i in [3, 6, 9, 12]:
#         dt = date(year, month, 1)
#         tmp = order[pd.date_range(end=dt, periods=i, freq='M')]
#         X['has_ord_pre_%s' % i] = (tmp > 0).sum(axis=1).values  # 前i个月有提货的天数
#         X['last_ord_pre_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values  # 前i个月距离上一次有提货的天数
#         X['first_ord_pre_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values  # 前i个月距离第一次有提货的天数
        
    # 分销天数特征（9个特征）>_<|||
#     for i in [3, 6, 9]:
#         dt = date(year, month, 1)
#         tmp = dis[pd.date_range(end=dt, periods=i, freq='M')]
#         X['has_dis_pre_%s' % i] = (tmp > 0).sum(axis=1).values  # 前i个月有分销的天数
#         X['last_dis_pre_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values  # 前i个月距离上一次有分销的天数
#         X['first_dis_pre_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values  # 前i个月距离第一次有分销的天数
        
    # 库存天数特征（6个特征）>_<|||
#     for i in [3, 6]:
#         dt = date(year, month, 1)
#         tmp = inv[pd.date_range(end=dt, periods=i, freq='M')]
#         X['has_inv_pre_%s' % i] = (tmp > 0).sum(axis=1).values  # 前i个月有库存的天数
#         X['last_inv_pre_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values  # 前i个月距离上一次有库存的天数
#         X['first_inv_pre_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values  # 前i个月距离第一次有库存的天数
        
    # 前12个月的提货量
    for i in range(1, 13):
        if month - i <= 0:
            start_dt = date(year - 1, month + 12 - i, 1)
        else:
            start_dt = date(year, month - i, 1)
        X['ord_pre_%s' % i] = order[pd.date_range(start_dt, periods=1, freq='M')].values.ravel()
        
    # 前9个月的分销量>_<|||
    for i in range(1, 10):
        if month - i <= 0:
            start_dt = date(year - 1, month + 12 - i, 1)
        else:
            start_dt = date(year, month - i, 1)
        X['dis_pre_%s' % i] = dis[pd.date_range(start_dt, periods=1, freq='M')].values.ravel()
       
    # 前6个月的库存量>_<|||
    for i in range(1, 7):
        if month - i <= 0:
            start_dt = date(year - 1, month + 12 - i, 1)
        else:
            start_dt = date(year, month - i, 1)
        X['inv_pre_%s' % i] = inv[pd.date_range(start_dt, periods=1, freq='M')].values.ravel()
        
    X = pd.DataFrame(X)
    
    if is_train:
        start_dt = date(year, month, 1)
        y = order[pd.date_range(start_dt, periods=3, freq='M')].values
        return X, y
    
    if name_prefix is not None:
        X.columns = ['%s_%s' % (name_prefix, c) for c in X.columns]
        
    return X

## 2.1 准备训练集

In [30]:
train_month = [ 
    '2018-01', 
    '2018-02', 
    '2018-03', 
    '2018-04'
]

X_l, y_l = [], []
for month in train_month:
    y, m = int(month.split('-')[0]), int(month.split('-')[1])
    
    X_tmp, y_tmp = prepare_dataset(order_month, dis_month, inv_month, y, m)
    
    X_tmp2 = prepare_dataset(order_cate_month, dis_cate_month, inv_cate_month, y, m, is_train=False, name_prefix='cate')
    X_tmp2.index = order_cate_month.index
    X_tmp2 = X_tmp2.reindex(category['category']).reset_index(drop=True)
    
    X_tmp = pd.concat([X_tmp, X_tmp2, category.reset_index(drop=True)], axis=1)
    X_l.append(X_tmp)
    y_l.append(y_tmp)
    
    del X_tmp, y_tmp, X_tmp2
    gc.collect()
    
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)

## 2.2 准备验证集

In [31]:
X_val, y_val = prepare_dataset(order_month, dis_month, inv_month, 2018, 7)

X_val2 = prepare_dataset(order_cate_month, dis_cate_month, inv_cate_month, 2018, 7, is_train=False, name_prefix='cate')
X_val2.index = order_cate_month.index
X_val2 = X_val2.reindex(category['category']).reset_index(drop=True)

X_val = pd.concat([X_val, X_val2, category.reset_index(drop=True)], axis=1)

del X_val2
gc.collect()

0

## 2.3 准备测试集

In [32]:
X_test = prepare_dataset(order_month, dis_month, inv_month, 2018, 10, is_train=False)

X_test2 = prepare_dataset(order_cate_month, dis_cate_month, inv_cate_month, 2018, 10, is_train=False, name_prefix='cate')
X_test2.index = order_cate_month.index
X_test2 = X_test2.reindex(category['category']).reset_index(drop=True)

X_test = pd.concat([X_test, X_test2, category.reset_index(drop=True)], axis=1)

del X_test2
gc.collect()

0

# 3 训练和预测

In [33]:
print("[INFO] Start training and predicting...")
t0 = time()

params = {
    'num_leaves': 80, 
    'objective': 'regression', 
    'min_data_in_leaf': 200, 
    'learning_rate': 0.01, 
    'feature_fraction': 0.9, 
    'bagging_fraction': 0.8, 
    'bagging_freq': 1, 
    'metric': 'l2', 
    'num_threads': 16
}

MAX_ROUNDS = 5000
pred_val = []
pred_test = []
cate_vars = []

for i in range(3):
    print('=' * 50)
    print("Step %d" % (i + 1))
    print('=' * 50)
    
    dtrain = lgb.Dataset(X_train, label=y_train[:,i], categorical_feature=cate_vars)
    dval = lgb.Dataset(X_val, label=y_val[:,i], reference=dtrain, categorical_feature=cate_vars)
    
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS, 
        valid_sets=[dtrain, dval], early_stopping_rounds=125, verbose_eval=50
    )
    
    feat_imp = [("%s: %.2f" % x) for x in sorted(zip(X_train.columns, bst.feature_importance('gain')), key=lambda x: x[1], reverse=True)]
    print('\n'.join(feat_imp))
    pred_val.append(
        bst.predict(X_val, num_iteration=bst.best_iteration or MAX_ROUNDS)
    )
    pred_test.append(
        bst.predict(X_test, num_iteration=bst.best_iteration or MAX_ROUNDS)
    )
    
print("[INFO] Finished! ( ^ _ ^ ) V")
print("[INFO] Done in %f seconds." % (time() - t0))

[INFO] Start training and predicting...
Step 1


/root/miniconda3/envs/xPy3/lib/python3.6/site-packages/lightgbm/basic.py:1184: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 125 rounds.
[50]	training's l2: 6.4699	valid_1's l2: 5.65617
[100]	training's l2: 4.35745	valid_1's l2: 4.19036
[150]	training's l2: 3.49002	valid_1's l2: 3.73468
[200]	training's l2: 3.10197	valid_1's l2: 3.63887
[250]	training's l2: 2.89668	valid_1's l2: 3.65655
[300]	training's l2: 2.765	valid_1's l2: 3.6925
Early stopping, best iteration is:
[210]	training's l2: 3.0522	valid_1's l2: 3.6298
ord_max_pre_3: 756303.71
ord_sum_decay_pre_3: 717502.35
ord_pre_2: 269157.07
ord_mean_pre_3: 129642.14
ord_pre_1: 84090.77
cate_ord_diff_mean_pre_6: 39430.67
cate_ord_diff_mean_pre_12: 36798.61
ord_diff_mean_pre_3: 16605.77
ord_median_pre_3: 16096.40
ord_min_pre_3: 14946.42
ord_diff_mean_pre_6: 12517.78
ord_sum_decay_pre_6: 10792.64
cate_dis_diff_mean_pre_6: 8801.16
ord_pre_4: 8739.10
ord_max_pre_6: 7203.13
cate_ord_diff_mean_pre_3: 5590.03
ord_std_pre_3: 5462.41
cate_ord_pre_4: 5164.71
cate_dis_pre_8: 5156.67
inv_pre_2: 4784.21
cate_dis_std_pre_

Training until validation scores don't improve for 125 rounds.
[50]	training's l2: 7.15949	valid_1's l2: 8.1368
[100]	training's l2: 5.55315	valid_1's l2: 6.56768
[150]	training's l2: 4.85515	valid_1's l2: 5.93794
[200]	training's l2: 4.51488	valid_1's l2: 5.70487
[250]	training's l2: 4.32146	valid_1's l2: 5.61378
[300]	training's l2: 4.19063	valid_1's l2: 5.63346
[350]	training's l2: 4.09511	valid_1's l2: 5.66687
Early stopping, best iteration is:
[259]	training's l2: 4.29572	valid_1's l2: 5.60418
ord_pre_1: 674945.18
ord_sum_decay_pre_3: 375528.36
ord_max_pre_3: 152113.23
ord_mean_pre_3: 59151.17
ord_sum_decay_pre_6: 44465.93
dis_mean_pre_9: 41864.69
ord_max_pre_6: 32090.77
ord_pre_2: 31386.70
ord_min_pre_3: 27405.58
ord_max_pre_9: 22980.06
cate_ord_diff_mean_pre_12: 16868.57
cate_dis_std_pre_6: 16814.82
ord_diff_mean_pre_12: 14472.44
dis_pre_8: 13396.89
dis_std_pre_9: 10713.53
cate_dis_std_pre_3: 8874.59
cate_ord_pre_12: 8798.19
inv_pre_2: 8539.54
cate_dis_diff_mean_pre_6: 7448.36
o

In [34]:
print("The MSE error of validation set is:", mean_squared_error(y_val, np.array(pred_val).transpose()))

The MSE error of validation set is: 4.574220182022851


评价指标为**归一化加权均方根对数误差（Normalized Weighted Root Mean Squared Logarithmic Error, NWRMSLE）**，表达式如下：

$$
\text{NWRMSLE} = \sqrt{ \frac{\sum_{i=1}^{m}\omega_i \sum_{j=1}^{t}\left( \ln \left(\widehat{y}_j^{(i)} + 1 \right) -\ln \left(y_j^{(i)} + 1 \right)\right)^2}{t\cdot \sum_{i=1}^{m}\omega_i} }
$$

In [35]:
# 归一化加权均方根对数误差
err = (y_val - np.array(pred_val).transpose()) ** 2
err = err.sum(axis=1)
err = np.sqrt(err.sum() / 3 / len(y_val))
print("The NWRMSLE error of validation set is:", err)

The NWRMSLE error of validation set is: 2.138742663815086


业务指标的表达式如下：

$$
E(t, y) = \frac{\sum_{i=0}^{n} | y^{(i)} - t^{(i)} |}{\sum_{i=0}^{n} t^{(i)}}
$$

In [36]:
def error(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(y_true)

In [37]:
def expm1_with_clip(x, l=0, r=100000):
    return np.clip(np.expm1(x), l, r)

In [38]:
df_val = pd.DataFrame(
    np.array(y_val), index=order_month.index, 
    columns=['2018-07', '2018-08', '2018-09']
)

In [39]:
df_pred_val = pd.DataFrame(
    np.array(pred_val).transpose(), index=order_month.index, 
    columns=['2018-07', '2018-08', '2018-09']
)

In [40]:
# 验证集准确率
first_month_error = error(np.expm1(df_val['2018-07']), np.expm1(df_pred_val['2018-07']))
second_month_error = error(np.expm1(df_val['2018-08']), np.expm1(df_pred_val['2018-08']))
third_month_error = error(np.expm1(df_val['2018-09']), np.expm1(df_pred_val['2018-09']))

In [41]:
print("The accuracy of 'M+1' order amount is: %.2f%%" % ((1 - first_month_error) * 100))
print("The accuracy of 'M+2' order amount is: %.2f%%" % ((1 - second_month_error) * 100))
print("The accuracy of 'M+3' order amount is: %.2f%%" % ((1 - third_month_error) * 100))

The accuracy of 'M+1' order amount is: 25.23%
The accuracy of 'M+2' order amount is: 16.68%
The accuracy of 'M+3' order amount is: 11.81%


In [42]:
df_pred_test = pd.DataFrame(
    np.array(pred_test).transpose(), index=order_month.index, 
    columns=['2018-10', '2018-11', '2018-12']
).stack().to_frame('pred_qty')
df_pred_test.index.set_names(['item_code', 'month'], inplace=True)
df_pred_test['pred_qty'] = np.clip(np.expm1(df_pred_test['pred_qty']), 0, 100000)

In [43]:
comp = df_test.join(df_pred_test, how='left').fillna(0).reset_index()
first_month_comp = comp.loc[comp['month'] == '2018-10']
second_month_comp = comp.loc[comp['month'] == '2018-11']
third_month_comp = comp.loc[comp['month'] == '2018-12']

In [44]:
first_month_error = error(first_month_comp['qty'], first_month_comp['pred_qty'])
second_month_error = error(second_month_comp['qty'], second_month_comp['pred_qty'])
third_month_error = error(third_month_comp['qty'], third_month_comp['pred_qty'])

In [45]:
print("The accuracy of 'M+1' order amount is: %.2f%%" % ((1 - first_month_error) * 100))
print("The accuracy of 'M+2' order amount is: %.2f%%" % ((1 - second_month_error) * 100))
print("The accuracy of 'M+3' order amount is: %.2f%%" % ((1 - third_month_error) * 100))

The accuracy of 'M+1' order amount is: 15.44%
The accuracy of 'M+2' order amount is: 19.37%
The accuracy of 'M+3' order amount is: 8.94%


# 4 规则微调

In [46]:
# 得到品类的上限取值（取品类每个月最大值的60%），用于最后微调结果
# cate_max = {}
# cates = list(order.category.unique())
# for cate in cates:
#     tmp = order.loc[order.category == cate]
#     tmp['month'] = tmp['order_date'].astype('str').apply(lambda x: x[:7])
#     tmp = tmp.groupby(['item_code', 'month'])[['qty']].sum()
#     cate_max[cate] = int(tmp.qty.max() * 0.6)